# 4.9 Part I

**import libraries**

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

**import dataframes**

In [2]:
path = r'C:\Users\Anwender\Desktop\Instacart Basket Analysis-Alessia'

In [4]:
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [5]:
ords_prods_group = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_updated_v2.pkl'))

## 4. Data Wrangling - new 'customer' data set

**4.1 checking data shape**

In [6]:
customers.shape

(206209, 10)

**4.2 checking info**

In [7]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


***comment: First Name non/null should have the same value as all the other columns. Less non-null should mean that, in fact, we have quite  abit of null cells in the First Name column. I am going to check that now. Also, the user_id should not be an integer but a string. Will change that later***

In [9]:
customers['First Name'].value_counts(dropna = False)

NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: First Name, Length: 208, dtype: int64

***comment: First name column has 11259 empty rows (NaN). Will need to address this in the next steps***

**4.3 checking names of the columns to find illogical names**

In [10]:
customers.columns

Index(['user_id', 'First Name', 'Surnam', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

***comment: I am going to change Surnam to Surname and STATE to State**

In [11]:
customers.rename(columns = {'Surnam' : 'Surname'}, inplace = True)

In [12]:
customers.rename(columns = {'STATE' : 'State'}, inplace = True)

**checking if changes have been applied**

In [13]:
customers.head()

,user_id,First Name,Surname,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## 5. Data quality and consistency checks

**5.1 check missing values**

In [14]:
customers.isnull().sum()

user_id             0
First Name      11259
Surname             0
Gender              0
State               0
Age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

***comment: First Name with 11259 empty cells***

**5.2 check duplicates**

In [15]:
df_dups = customers[customers.duplicated()]

In [16]:
df_dups

,user_id,First Name,Surname,Gender,State,Age,date_joined,n_dependants,fam_status,income


***comment: no duplicates found***

**5.3 convert mixed-type data**

In [17]:
for col in customers.columns.tolist():
    weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (customers[weird]) > 0:
        print(col)

First Name


***comment: seems that First name column has mixed type data. It should all be a string, so I will change the data type to string.***

In [18]:
customers['First Name'] = customers['First Name'].astype('str')

***comment: also changing user_id to string***

In [19]:
customers['user_id'] = customers['user_id'].astype('str')

In [20]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  object
 1   First Name    206209 non-null  object
 2   Surname       206209 non-null  object
 3   Gender        206209 non-null  object
 4   State         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 15.7+ MB


***all changes have been correctely applied***

## 6. Combine customer data with the rest of the prepared Instacart data

**6.1 check orders file**

In [21]:
ords_prods_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 26 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   orders_day_of_week      int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   product_id              int64   
 7   add_to_cart_order       int64   
 8   reordered               int64   
 9   Unnamed: 0_y            int64   
 10  product_name            object  
 11  aisle_id                int64   
 12  department_id           int64   
 13  prices                  float64 
 14  _merge                  category
 15   price_range_loc        object  
 16  price_range_loc         object  
 17  busiest_day             object  
 18  busiest_days            object  
 19  busiest_period_of_day   object  
 20  max_order               int64   
 21  loyalt

In [22]:
ords_prods_group.shape

(32404859, 26)

***Comment: I want to combine the two files on user_id, but they have two different data types. I am changing the user_id of the ords_prods_group into a string***

In [23]:
ords_prods_group ['user_id'] = ords_prods_group ['user_id'].astype('str')

In [24]:
ords_prods_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 26 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 object  
 2   order_number            int64   
 3   orders_day_of_week      int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   product_id              int64   
 7   add_to_cart_order       int64   
 8   reordered               int64   
 9   Unnamed: 0_y            int64   
 10  product_name            object  
 11  aisle_id                int64   
 12  department_id           int64   
 13  prices                  float64 
 14  _merge                  category
 15   price_range_loc        object  
 16  price_range_loc         object  
 17  busiest_day             object  
 18  busiest_days            object  
 19  busiest_period_of_day   object  
 20  max_order               int64   
 21  loyalt

**6.2 combining the two dataframes**

In [25]:
combined = ords_prods_group.merge(customers, on = 'user_id')

**6.3 checking new dataframe**

In [26]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 35 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 object  
 2   order_number            int64   
 3   orders_day_of_week      int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   product_id              int64   
 7   add_to_cart_order       int64   
 8   reordered               int64   
 9   Unnamed: 0_y            int64   
 10  product_name            object  
 11  aisle_id                int64   
 12  department_id           int64   
 13  prices                  float64 
 14  _merge                  category
 15   price_range_loc        object  
 16  price_range_loc         object  
 17  busiest_day             object  
 18  busiest_days            object  
 19  busiest_period_of_day   object  
 20  max_order               int64   
 21  loyalt

In [27]:
combined.shape

(32404859, 35)

In [28]:
combined.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,...,frequency_flag,First Name,Surname,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,196,1,0,195,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,195,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,195,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,195,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,195,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


***comment: seems everything ok, but the column Unnamed: 0_y appeared. I am deleting it***

In [29]:
combined = combined.drop(columns = ['Unnamed: 0_y'])

In [30]:
combined.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'customer_ords_prods_merged.pkl'))